In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType, DataType,ArrayType
from pyspark.sql.functions import udf, struct
from pyspark.ml import Pipeline
from IPython.display import display, HTML
import re

In [2]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
# spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[2]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.3,org.postgresql:postgresql:42.5.0")\
    .getOrCreate()
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

22/11/21 14:08:07 WARN Utils: Your hostname, Glorias-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.39 instead (on interface en0)
22/11/21 14:08:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/gloria/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/gloria/.ivy2/cache
The jars for the packages stored in: /Users/gloria/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4d0921f2-e2a3-4bf4-90d4-fdb281b08878;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastu

22/11/21 14:08:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/21 14:08:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark NLP version 4.2.0
Apache Spark version: 3.3.0


In [3]:
### bible keyword frequency
df_keyword=spark.read.json('./bibleKeyword.json/')          
df_keyword.show()
df_keyword=df_keyword.selectExpr('text','keywords.result as keywords')                  
df_keyword.show()                                                              
            

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            document|highlighted_keywords|            keywords|            sentence|                text|               token|     unique_keywords|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[{document, 0, []...|In the <span styl...|[{chunk, 7, [], 1...|[{document, 0, []...|In the beginning ...|[{token, 0, [], 1...|[beginning, god, ...|
|[{document, 0, []...|Now the <span sty...|[{chunk, 8, [], 1...|[{document, 0, []...|Now the earth was...|[{token, 0, [], 2...|[earth, formless,...|
|[{document, 0, []...|And <span style="...|[{chunk, 4, [], 6...|[{document, 0, []...|And God said, "Le...|[{token, 0, [], 2...|[god, said, light...|
|[{document, 0, []...|<span style="back...|[{chunk, 0, [], 2...|[{document, 0, []...|God saw that the ...|

In [4]:

from pyspark.sql.functions import explode
df2 = df_keyword.select('text',explode(df_keyword.keywords).alias('keyword'))
df2.printSchema()
df2.show()



root
 |-- text: string (nullable = true)
 |-- keyword: string (nullable = true)

+--------------------+--------------------+
|                text|             keyword|
+--------------------+--------------------+
|In the beginning ...|           beginning|
|In the beginning ...|                 god|
|In the beginning ...|             created|
|In the beginning ...|             heavens|
|In the beginning ...|               earth|
|In the beginning ...|       beginning god|
|In the beginning ...|         god created|
|In the beginning ...|beginning god cre...|
|In the beginning ...| created the heavens|
|Now the earth was...|               earth|
|Now the earth was...|            formless|
|Now the earth was...|               empty|
|Now the earth was...|            darkness|
|Now the earth was...|             surface|
|Now the earth was...|                deep|
|Now the earth was...|              spirit|
|Now the earth was...|                 god|
|Now the earth was...|            hover

In [5]:
df2.count()
df_keyword=df2.select('keyword').groupBy('keyword').count()



DataFrame[summary: string, keyword: string, count: string]

In [6]:
df_keyword.describe().show()

+-------+-------+------------------+
|summary|keyword|             count|
+-------+-------+------------------+
|  count|  96024|             96024|
|   mean|   null|4.4531263017578935|
| stddev|   null| 42.76291857387599|
|    min|  aaron|                 1|
|    max|zuzites|              7144|
+-------+-------+------------------+



In [7]:
df_keyword.orderBy('count',ascending=False).show()
# df_keyword.printSchema()
# df_keyword.createTempView('keyword')     
# df_keyword.describe().show()

+-------+-----+
|keyword|count|
+-------+-----+
|   lord| 7144|
|    god| 3691|
|   said| 3019|
|    one| 2410|
|   king| 2226|
|    son| 2170|
| people| 2064|
|    man| 1987|
| israel| 1701|
|    men| 1687|
|   like| 1503|
|   land| 1362|
|    day| 1308|
|   come| 1286|
|     us| 1279|
|  jesus| 1217|
|   went| 1213|
|   came| 1173|
|     go| 1153|
|    may| 1131|
+-------+-----+
only showing top 20 rows



In [8]:
df_keyword.write.format("jdbc")\
    .option("url", "jdbc:postgresql://192.168.1.39:5432/aiknowledge")\
    .option("dbtable", "public.bible_keyword")\
    .option("user", "postgres")\
    .option("password", "postgres")\
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite').save()

### write to neo4j

from neo4j import GraphDatabase
import time
from tqdm import tqdm

In [19]:
from py2neo import Graph
from neo4j import GraphDatabase

graph = Graph("neo4j://localhost:7687", auth=("daniel", "fighting"),name="aiknowledge")
graph.run("UNWIND range(1, 3) AS n RETURN n, n * n as n_sq")

n,n_sq
1,1
2,4
3,9


In [20]:
df_keyword.show()

+------------------+-----+
|           keyword|count|
+------------------+-----+
|            waters|  157|
|         first day|   48|
|  noah and entered|    2|
|        great deep|    4|
|             still|  278|
|        may canaan|    2|
|          ashkenaz|    3|
|men moved eastward|    1|
|             serug|    6|
|            spared|   24|
| anything too hard|    2|
|    sake of twenty|    1|
|      two men said|    1|
|        swept away|   10|
|         ammonites|   84|
|              hazo|    1|
|       master made|    1|
|           jewelry|   12|
|         go toward|    1|
|        loved esau|    1|
+------------------+-----+
only showing top 20 rows



In [27]:
from py2neo import Graph, Node, Relationship
import time
from tqdm import tqdm

for row in tqdm(df_keyword.rdd.collect()):
    keyword = Node('bible_keyword', name=row['keyword'])
    keyword['count']=row['count']
    graph.create(keyword)


  

100%|██████████| 96024/96024 [12:08<00:00, 131.85it/s]
